# Visualize HDF5 in a knowledge graph

Using `kglab` (https://derwen.ai/docs/kgl/)

In [1]:
# !pip install kglab

In [2]:
import kglab
import rdflib
import numpy as np

In [3]:
import h5rdmtoolbox as h5tbx

In [12]:
with h5tbx.File() as h5:
    h5.create_dataset(name='test', data=4.3)
    h5.create_dataset(name='nd-array', data=np.random.rand(2, 3), chunks=(1,3))
    grp = h5.create_group(name='contact')
    grp.attrs['fname', rdflib.FOAF.firstName] = 'Matthias'
    grp.attrs['lname', rdflib.FOAF.lastName] = 'Probst'
    grp.attrs['hint'] = 'This group could be representing a person.'
    grp.rdf['hint'].definition = 'A hint gives helpful information on something.'
    grp.rdf.type = rdflib.PROV.Person  # --> rdf.type = Person
    grp.rdf.subject = 'https://orcid.org/0000-0001-8729-0482'  # -> @id='https://orcid.org/0000-0001-8729-0482'

    print(h5.dump_jsonld(indent=2, structural=True, file_uri="https://example.org#", context=dict(ex="https://example.org#")))
    
graph = h5tbx.get_ld(h5.hdf_filename)

{
  "@context": {
    "dcterms": "http://purl.org/dc/terms/",
    "ex": "https://example.org#",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "hdf": "http://purl.allotrope.org/ontologies/hdf5/1.8#",
    "prov": "http://www.w3.org/ns/prov#",
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  },
  "@graph": [
    {
      "@id": "ex:tmp2.hdf/nd-array__dataspace_dimension_0",
      "@type": "hdf:DataspaceDimension",
      "hdf:dimensionIndex": 0,
      "hdf:size": 2
    },
    {
      "@id": "ex:tmp2.hdf/test__dataspace",
      "@type": "hdf:ScalarDataspace"
    },
    {
      "@id": "ex:tmp2.hdf/contact",
      "@type": "hdf:Group",
      "dcterms:relation": {
        "@id": "https://orcid.org/0000-0001-8729-0482"
      },
      "hdf:attribute": [
        {
          "@id": "ex:tmp2.hdf/contact@hint"
        },
        {
          "@id": "ex:tmp2.hdf/contact@lname"
        },
        {
          "@id": "ex:tmp2.hdf/contact@fname"
        }
      ],
      "hdf:name": "/contact"
   

In [13]:
kg = kglab.KnowledgeGraph(
    import_graph=graph,
    namespaces={k: str(v) for k, v in dict(graph.namespaces()).items()})

In [14]:
subgraph = kglab.SubgraphTensor(kg)

In [15]:
VIS_STYLE= {
    "hdf": {"color": "red", "size": 30},
    "_": {"color": "black", "size": 20}
}

In [16]:
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.show('graph.html', notebook=True)
# the above call fails in the readthedocs build process... we show an image of the result here:

graph.html


![grafik.png](../_static/Screenshot_pyvis_graph.png)

In [17]:
with h5tbx.File() as h5:
    h5.create_dataset(name='test', data=4.3)
    grp = h5.create_group(name='contact')
    grp.attrs['fname', rdflib.FOAF.firstName] = 'Matthias'
    grp.attrs['lname', rdflib.FOAF.lastName] = 'Probst'
    grp.attrs['hint'] = 'bah blah'
    grp.rdf['hint'].definition = 'A hint'
    grp.rdf.subject = rdflib.PROV.Person
    grp.attrs['@id'] = 'https://orcid.org/0000-0001-8729-0482'
    print(h5.dump_jsonld(indent=2, structural=False))

graph = h5tbx.get_ld(h5.hdf_filename, structural=False)

{
  "@context": {
    "dcterms": "http://purl.org/dc/terms/",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "prov": "http://www.w3.org/ns/prov#"
  },
  "@graph": [
    {
      "@id": "prov:Person",
      "foaf:firstName": "Matthias",
      "foaf:lastName": "Probst"
    },
    {
      "@id": "_:tmp3.hdf/contact",
      "dcterms:relation": {
        "@id": "prov:Person"
      }
    }
  ]
}


In [18]:
VIS_STYLE= {
    "hdf": {"color": "red", "size": 30},
    "_": {"color": "black", "size": 20},
    "prof": {"color": "green", "size": 10},
    "foaf": {"color": "green", "size": 10}
}

In [19]:
kg_from_jsonld = kglab.KnowledgeGraph().load_rdf_text(
    graph.serialize(format="ttl")
)

subgraph = kglab.SubgraphTensor(kg_from_jsonld)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)

pyvis_graph.force_atlas_2based(overlap=0)
pyvis_graph.show("tmp.grpah.html", local=True, notebook=True)
# the above call fails in the readthedocs build process... we show an image of the result here:

tmp.grpah.html


![grafik.png](../_static/Screenshot_pyvis_graph2.png)